In [107]:
%pylab inline
from glob import glob
import math
from PIL import Image
import json
import sklearn.manifold
import os

def resize_to_power_of_two(nSize):
    pos =  math.ceil(math.log2(nSize))  #(ceiling of log n with base 2)
    p   = pow(2, pos) 
    return p

def generate_spritesheet(fnames,photoSize,filename,positions,saveImage):
    """\
    Make a contact sheet from a group of filenames:

    fnames       A list of names of the image files
    
    ncols        Number of columns in the contact sheet
    nrows        Number of rows in the contact sheet
    photow       The width of the photo thumbs in pixels
    photoh       The height of the photo thumbs in pixels

    returns a PIL image object.
    """

    # Calculate the size of the output image, based on the photo thumb sizes
    
    totalNum = len(fnames)
    
    ncols = 64
    nrows = math.ceil(totalNum/ncols)
    
    photow = photoSize[0]
    phtoh = photoSize[1]
   
    
    imgWidth = ncols*photow
    imgHeight = nrows*photoh
    
    imgWidth_resized = resize_to_power_of_two(imgWidth)
    imgHeight_resized = resize_to_power_of_two(imgHeight)
    
    isize = (imgWidth_resized,imgHeight_resized) # adapt to THREE

    # Create the new image. The background doesn't have to be white
    white = (0,0,0)
    inew = Image.new('RGB',isize,white)
    
    # Create JSON file to store the UV position of each image
    uvDict = {}
    uvDict['rows'] = nrows
    uvDict['columns'] = ncols
    uvDict['totalCount'] = totalNum
    uvDict['spriteWidth'] = photow
    uvDict['spriteHeight'] = phtoh
    count = 0
    # Insert each thumb:
    for irow in range(nrows):
        for icol in range(ncols):
            left = icol*(photow)
            right = left + photow
            upper = irow*(photoh)
            lower = upper + photoh
            bbox = (left,upper,right,lower)
            try:
                # Read in an image and resize appropriately
                img = Image.open(fnames[count]).resize((photow,photoh))
            except:
                break
            inew.paste(img,bbox)
            if count not in uvDict.keys():
                uvDict[count] = {}
            uvDict[count]['filename'] = os.path.basename(fnames[count])
            uvDict[count]['uvOffset_u'] = left/imgWidth
            uvDict[count]['uvOffset_v'] = 1.0-lower/imgHeight_resized 
            uvDict[count]['uvRepeat_u'] = photow/imgWidth_resized
            uvDict[count]['uvRepeat_v'] = photoh/imgHeight_resized
            uvDict[count]['position'] = {}
            uvDict[count]['position']['x'] = str(positions[count][0])
            uvDict[count]['position']['y'] = str(positions[count][1])
            if len(positions[count]) > 2:
                uvDict[count]['position']['z'] = str(positions[count][2])
            else:
                uvDict[count]['position']['z'] = str(0.0)
            count += 1
    js = json.dumps(uvDict, indent=2)
    f = open(filename + '.json', 'w')
    f.write(js)
    f.close()
    if saveImage:
        inew.save(filename)
    return inew

Populating the interactive namespace from numpy and matplotlib


In [110]:
gamedir = './pic/Super Mario World (USA)/'
files = glob(gamedir + 'screenshots/*.png')
files = sorted(files, key=lambda x: int(os.path.basename(x)[:-4]))

embeddings = glob(gamedir + '/embeddings/*.npy')
embeddings = sorted(embeddings, key=lambda x: int(os.path.basename(x)[:-4]))

length = len(embeddings)

states = zeros((length, 256))
for i in range(len(states)):
    states[i] = load(embeddings[i])
    
tsne = sklearn.manifold.TSNE(n_components=3, perplexity = 400)
positions = tsne.fit_transform(states)
print(positions.shape, end='\r')

photow,photoh = 64,56
photoSize = (photow,photoh)
filename = './pic/spritesheet_400.png'


In [111]:
saveImage = False
inew = generate_spritesheet(files,photoSize,filename,positions,saveImage)
inew.show()

In [84]:
embeddings[0]

'./pic/Super Mario World (USA)//embeddings/10.npy'

In [88]:
shape(states[0])

(256,)